# NUAGE COMPTA ET LA FACTURE S'ANALYSA
# Création du modèle

## 0. Import des librairies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import keras
from keras import metrics
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation
import os
import pickle
import cv2
import imutils
from imutils.contours import sort_contours

from src import fonctions

Using TensorFlow backend.


## 1. Optimisation et nettoyage des données

In [2]:
### Afin de réduire les temps d'attente et dans un soucis d'optimisation, je decide d'importer une fonction qui prend 200 png par lettres (suffisant ici). Ces images viennent s'ajouter au dossier dataset_reduit_train ###
fonctions.import_fichier_train()

In [3]:
### Afin de réduire les temps d'attente et dans un soucis d'optimisation, je decide d'importer une fonction qui prend 3 png par lettres (suffisant ici). Ces images viennent s'ajouter au dossier dataset_reduit_test ###
fonctions.import_fichier_test()

## 2. Import des Datasets

In [4]:
### ImageDataGenerator génère des lots de données d'image vectorielles, convertissant les coefficients RVB compris entre 0 et 255 en valeurs cibles comprises entre 0 et 1 par mise à l'échelle avec un facteur de 1/255 à l' aide de la remise à l' échelle ###

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    ### shear_range est utilisé pour appliquer de manière aléatoire des transformations de cisaillement ###
    shear_range = 0.2,
    ### zoom_range est utilisé pour zoomer aléatoirement à l'intérieur des images ###           
    zoom_range = 0.2,
    ### horizontal_flip est utilisé pour retourner au hasard la moitié des images horizontalement ###            
    horizontal_flip = True
)      

test_datagen = ImageDataGenerator(rescale = 1./255)


### J'importe les images une par une à partir des répertoires en utilisant .flow_from_directory et y appliquons ImageDataGenerator ###

train_generator = train_datagen.flow_from_directory(
    ### Choix de mon repertoire ###
    directory = "data/dataset_reduit_train",
    ### Je converti les images de leur taille d'origine à notre target_size ###                    
    target_size = (28,28),
    ### Nombre batch_size qui fait référence au nombre d'exemples d'entraînement utilisés dans une itération ###                                      
    batch_size = 32,
    ### Je definis le class_mode sur "catégorical" indiquant que nous avons plusieurs classes (a à z) à prédire ###          
    class_mode = "categorical",
    ### Je choisis le color_mode "grayscale", indiquant que nous trvaillons sur une image en noir et blanc
    color_mode = "grayscale"                                

)

test_generator = test_datagen.flow_from_directory(
    directory = "data/dataset_reduit_test",
    target_size = (28,28),
    batch_size = 32,
    class_mode = "categorical",
    color_mode = "grayscale"

)

Found 5206 images belonging to 27 classes.
Found 1566 images belonging to 27 classes.


## 3. Definition du modèle d'architecture

In [5]:
### Création d'un modèle séquentiel qui permet de définir l'architecture CNN couche par couche à l'aide de la fonction .add .Nous ajoutons d'abord une couche de convolution avec 32 filtres de taille 3X3 sur les images d'entrée et la passons à travers la fonction d'activation 'relu'.Nous effectuons ensuite des opérations MaxPooling en utilisant un pool de taille 2X2 ###
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = (28,28,1), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

### Ces couches sont ensuite répétées à nouveau pour améliorer les performances du modèle ###

model.add(Conv2D(32, (3, 3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

### Enfin, nous aplatissons notre matrice résultante et la passons à travers une couche dense composée de 128 nœuds. Celui-ci est ensuite connecté à la couche de sortie constituée de 26 nœuds, chaque nœud représentant un alphabet ###

model.add(Flatten())
model.add(Dense(units = 128, activation = "relu"))
model.add(Dense(units = 27, activation = "softmax"))            ### Activation softmax qui convertit les scores en une distribution de probabilité normalisée, et                                                                   le nœud avec la probabilité la plus élevée est sélectionné comme sortie ###

### Une fois notre architecture CNN définie, nous compilons le modèle à l'aide de l'optimiseur Adam ###
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               102528    
_________________________________________________________________
dense_2 (Dense)              (None, 27)               

## 4. Entrainement et enregistrement du modèle

In [6]:
### Je décide de créer 25 répétitions, et j'ai X2 les steps_per_epoch pour augmenter notre précision ###

entrainement = model.fit_generator(
    train_generator,
    steps_per_epoch = 32,
    epochs = 35,
    validation_data = test_generator,
    validation_steps = 32
)

score = model.evaluate(train_generator, verbose=0)
print("Test de perte:", score[0])
print("Test de précision:", score[1])
print("Enregistrement du modèle...")
model.save("data/model.h5")
print("Modèle enregistré!")


Epoch 1/35
32/32 [==============================] - 18s 552ms/step - loss: 3.1376 - accuracy: 0.1223 - val_loss: 2.7737 - val_accuracy: 0.2969
Epoch 2/35
32/32 [==============================] - 9s 294ms/step - loss: 2.2557 - accuracy: 0.3760 - val_loss: 1.5515 - val_accuracy: 0.4501
Epoch 3/35
32/32 [==============================] - 7s 225ms/step - loss: 1.7654 - accuracy: 0.4707 - val_loss: 1.0521 - val_accuracy: 0.6064
Epoch 4/35
32/32 [==============================] - 8s 262ms/step - loss: 1.3874 - accuracy: 0.5713 - val_loss: 1.3732 - val_accuracy: 0.6360
Epoch 5/35
32/32 [==============================] - 9s 270ms/step - loss: 1.2274 - accuracy: 0.6211 - val_loss: 0.8561 - val_accuracy: 0.6595
Epoch 6/35
32/32 [==============================] - 2s 75ms/step - loss: 1.1207 - accuracy: 0.6553 - val_loss: 1.3988 - val_accuracy: 0.7148
Epoch 7/35
32/32 [==============================] - 2s 64ms/step - loss: 1.0276 - accuracy: 0.6865 - val_loss: 0.5429 - val_accuracy: 0.6977
Epoch 8